### Importação das Bibliotecas
Carregamento dos pacotes necessários para manipulação de dados, análise estatística, visualização e construção dos modelos de machine learning.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy import stats
from scipy.stats import norm

from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from scipy.special import boxcox1p

from sklearn.linear_model import ElasticNet, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

In [ ]:
# Configurações iniciais
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Carregamento dos dados
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Inspeção das dimensões
print(f"Dimensões do Treino: {train_df.shape}")
print(f"Dimensões do Teste: {test_df.shape}")

# Verificação inicial
display(train_df.head())
print(train_df.dtypes.value_counts())

### Análise da Variável Alvo e Outliers
<p>Visualização da distribuição da variável SalePrice para verificação de normalidade e assimetria.</p>
<p>Em seguida, análise bivariada entre área habitável (GrLivArea) e preço para identificação e remoção de outliers, conforme recomendações da documentação do dataset.</p>

In [ ]:
# Configuração visual
sns.set_style("whitegrid")

# Visualizando a distribuição da variável alvo
plt.figure(figsize=(12, 5))

# Histograma com curva normal ajustada
plt.subplot(1, 2, 1)
sns.histplot(train_df['SalePrice'], kde=True, stat="density")
(mu, sigma) = norm.fit(train_df['SalePrice'])
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
plt.plot(x, norm.pdf(x, mu, sigma), 'k', linewidth=2)
plt.title(f'Distribuição SalePrice\n(Skewness: {train_df["SalePrice"].skew():.2f})')

# Q-Q Plot para verificar normalidade
plt.subplot(1, 2, 2)
stats.probplot(train_df['SalePrice'], plot=plt)
plt.tight_layout()
plt.show()

# Outliers
plt.figure(figsize=(8, 5))
sns.scatterplot(x=train_df['GrLivArea'], y=train_df['SalePrice'])
plt.axvline(x=4000, color='r', linestyle='--', label='Corte (> 4000 sq ft)')
plt.title('GrLivArea vs SalePrice')
plt.legend()
plt.show()

# Removendo casas com mais de 4000 sq ft e preço baixo
outliers_idx = train_df[(train_df['GrLivArea'] > 4000) & (train_df['SalePrice'] < 300000)].index
print(f"Removendo {len(outliers_idx)} outliers (IDs: {train_df.loc[outliers_idx, 'Id'].values})...")

train_df = train_df.drop(outliers_idx)
train_df.reset_index(drop=True, inplace=True)
print(f"Novas dimensões do treino: {train_df.shape}")

### Pré-processamento e Tratamento de Dados

Nesta etapa, aplicamos a transformação logarítmica na variável alvo (`SalePrice`) para normalizar sua distribuição. Em seguida, concatenamos os dados de treino e teste para garantir consistência no tratamento de valores ausentes.

A estratégia de imputação adotada baseia-se na natureza de cada variável:
- **Ausência Física:** Valores nulos em variáveis como piscina ou garagem são preenchidos com "None" (categóricas) ou 0 (numéricas).
- **Contexto Espacial:** `LotFrontage` é preenchida com a mediana da vizinhança.
- **Dados Faltantes Gerais:** Utilização da moda para as demais variáveis categóricas.
- **Limpeza:** Remoção da coluna `Utilities` por apresentar variância quase nula.

In [ ]:
# Transformação Logarítmica na Variável Alvo
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])
print("Log-transformation aplicada em SalePrice.")

# Concatenação de Treino e Teste
ntrain = train_df.shape[0]
ntest = test_df.shape[0]
y_train = train_df.SalePrice.values

# Remove Id e SalePrice para criar o dataset 'all_data'
all_data = pd.concat((train_df.drop(['Id', 'SalePrice'], axis=1), 
                      test_df.drop(['Id'], axis=1))).reset_index(drop=True)

print(f"Dimensão de all_data antes da limpeza: {all_data.shape}")

# Imputação de Valores Ausentes

# NA significa "Ausência" (Categoria 'None')
cols_none = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 
             'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
             'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
             'MasVnrType']

for col in cols_none:
    all_data[col] = all_data[col].fillna("None")

# NA significa "Ausência" (Valor 0 para numéricos)
cols_zero = ['GarageYrBlt', 'GarageArea', 'GarageCars',
             'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 
             'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea']

for col in cols_zero:
    all_data[col] = all_data[col].fillna(0)

# Preencher com a mediana da vizinhança
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

# Imputação pela moda
cols_mode = ['MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 
             'Exterior2nd', 'SaleType']

for col in cols_mode:
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

# Casos Específicos
all_data["Functional"] = all_data["Functional"].fillna("Typ") # Assumido Típico
all_data = all_data.drop(['Utilities'], axis=1) # Variável quase constante

# Verificação Final
total_null = all_data.isnull().sum().sum()
print(f"Total de valores nulos restantes: {total_null}")
print(f"Dimensão de all_data após limpeza: {all_data.shape}")

In [ ]:
# Ajuste de Tipos de Dados
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

# Label Encoding para Variáveis Ordinais
cols_ordinal = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
                'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 
                'BsmtFinType1', 'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 
                'GarageFinish', 'LandSlope', 'LotShape', 'PavedDrive', 'Street', 
                'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 'YrSold', 'MoSold')

print("Aplicando Label Encoding...")
for c in cols_ordinal:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# Criação de Novas Features
# A área total da casa
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
print(f"Variável 'TotalSF' criada.")

# Transformação de Variáveis Numéricas Assimétricas
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Calcula a assimetria de todas as variáveis numéricas
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})

# Filtra aquelas com assimetria alta (> 0.75)
skewness = skewness[abs(skewness) > 0.75]
print(f"Total de variáveis numéricas transformadas (Box-Cox): {skewness.shape[0]}")

# Suaviza a distribuição
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    all_data[feat] = boxcox1p(all_data[feat], lam)

# Transforma as variáveis categóricas nominais restantes em colunas binárias
all_data = pd.get_dummies(all_data)
print(f"Dimensão final do dataset: {all_data.shape}")

### Engenharia de Atributos e Transformação de Variáveis

Nesta etapa, realizamos modificações cruciais nas features para adequá-las aos modelos de regressão:

1.  **Ajuste de Tipos:** Conversão de variáveis numéricas que funcionam como categorias (como Ano e Mês de venda) para string.
2.  **Label Encoding:** Transformação de variáveis categóricas ordinais em números, preservando a hierarquia de valor (ex: qualidade dos materiais).
3.  **Criação de Features:** Geração da variável `TotalSF` (Área Total), somando as áreas do porão e dos andares habitáveis.
4.  **Tratamento de Assimetria:** Aplicação da transformação de Box-Cox em variáveis numéricas com alta assimetria (> 0.75) para aproximá-las de uma distribuição normal.
5.  **Dummy Variables:** Conversão final das variáveis categóricas nominais restantes em colunas binárias (One-Hot Encoding).

In [ ]:
# Preparação Final dos Dados
# Garante que ntrain esteja sincronizado com o y_train
ntrain = len(y_train)

# Separa o dataset 'all_data' de volta em treino e teste
train = all_data[:ntrain]
test = all_data[ntrain:]

print(f"Dimensões confirmadas -> Treino: {train.shape}, Target: {y_train.shape}")

# Configuração da Validação Cruzada
# K-Fold com 5 divisões
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    # A métrica de erro negativo é convertida para RMSE positivo
    rmse = np.sqrt(-cross_val_score(model, train.values, y_train, 
                                    scoring="neg_mean_squared_error", cv=kf))
    return(rmse)

# Definição dos Modelos
# Lasso para descartar features inúteis
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=1))

# ElasticNet
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

# Kernel Ridge
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

# Gradient Boosting
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state=5)

# Execução da Validação
models = {'Lasso': lasso, 'ElasticNet': ENet, 'KernelRidge': KRR, 'GradientBoosting': GBoost}

print("\n--- Performance dos Modelos (RMSE Log - Quanto menor, melhor) ---")
for name, model in models.items():
    score = rmsle_cv(model)
    print(f"{name}: {score.mean():.4f} (Desvio: {score.std():.4f})")

### Treinamento Final e Geração da Submissão

Com base nos resultados da validação cruzada, o modelo Lasso foi selecionado por apresentar o melhor desempenho (menor RMSE) e simplicidade.

Nesta etapa final, realizamos os seguintes procedimentos:
1.  **Treinamento:** O pipeline do modelo é ajustado utilizando todo o conjunto de dados de treino disponível.
2.  **Previsão:** Inferência dos preços sobre o conjunto de teste.
3.  **Reversão da Escala:** Aplicação da função exponencial (`np.expm1`) para reverter a transformação logarítmica realizada no início do projeto, retornando os valores para a escala monetária original.
4.  **Exportação:** Consolidação dos resultados no arquivo `submission.csv`, contendo os identificadores e as previsões de preço.

In [ ]:
# Definição e Treinamento do Modelo Final
# Lasso, pois apresentou a melhor performance na validação
final_model = make_pipeline(RobustScaler(), Lasso(alpha=0.0005, random_state=1))

print("Treinando o modelo final em todos os dados de treino...")
final_model.fit(train, y_train)

# Previsão nos Dados de Teste
print("Realizando previsões no dataset de teste...")
predictions_log = final_model.predict(test)

# Reversão da Escala Logarítmica
predictions_real = np.expm1(predictions_log)

# Geração do DataFrame de Submissão
test_df_original = pd.read_csv('data/test.csv') # Lendo apenas para garantir os IDs corretos
submission = pd.DataFrame()
submission['Id'] = test_df_original['Id']
submission['SalePrice'] = predictions_real

# Salvando o Arquivo CSV
submission.to_csv('submission.csv', index=False)
print("Arquivo 'submission.csv' gerado com sucesso!")

# Visualizando as primeiras linhas
print(submission.head())